In [1]:
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent

In [2]:
config_list=[
    {"model":"Qwen3",
     "base_url":"http://localhost:11434/v1",
     "api_key":"ollama",
     "price":[0,0]},
]
llm_config={'config_list':config_list}

In [3]:
from typing import Annotated, Literal

Operator= Literal["+","-","*","/"]

def calculator(a:int, b:int, operator:Annotated[Operator,"operator"]) -> int:
    if operator=="+":
        return a + b
    elif operator=="-":
        return a - b
    elif operator=="*":
        return a * b
    elif operator=="/":
        return int(a / b)
    else:
        raise ValueError(f"Unknown operator {operator}")

In [4]:
#let's define the assistant agent that suggests tools calls.
assistant=ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. you can help with simple calculation. Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

In [5]:
#The user proxy agent is used for interacting with assistant agent and execute tool calls.
user=ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg:msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [6]:
#Register the tool signature with assistant agent
assistant.register_for_llm(name="calculator",description="A simple calculator")(calculator)

#Register the tool function with the user proxy agent.
user.register_for_execution(name="calculator")(calculator)

In [7]:
from autogen import register_function
#Register the calculator function to two agent
register_function(
    calculator,
    caller=assistant, #the assistant agent can suggest calls to the calculator.
    executor=user,#the user proxy agent can execute the calculator calls.
    name="Calculator", #by default the function name is used as the tool name.
    description="A simple calculator",#A description of the tool
)

In [9]:
import math

def evaluate_expression(expression: str) -> str:
    """Evaluate a math expression safely and return the result."""
    try:
        # Define a safe set of functions/names
        allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith("__")}
        # Evaluate with limited globals
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"


In [12]:

register_function(
    evaluate_expression,
    caller=assistant,
    executor=user,
    description="Evaluate a math expression safely",
)


In [13]:
result = user.initiate_chat(
    assistant,
    message="What is (44232+13312/(232-32))*5?",
)
print(result)


User (to Assistant):

What is (44232+13312/(232-32))*5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

<think>
Okay, let's see. The user is asking for the result of this expression: (44232 + 13312/(232 - 32)) * 5. Hmm, I need to calculate this step by step. First, I should handle the operations inside the parentheses. The denominator here is 232 - 32. Let me compute that first. 232 minus 32 is 200. So now the expression becomes (44232 + 13312/200) * 5.

Next, I need to do the division: 13312 divided by 200. Let me calculate that. 13312 divided by 200 equals 66.56. So now the expression is (44232 + 66.56) * 5. Adding those together gives 44298.56. Then multiply by 5. Let me check that multiplication: 44298.56 * 5 equals 221,492.8. 

Wait, but maybe I should use the evaluate_expression function to make sure it's done correctly, especially since there are multiple operations and order of operations matt